In [1]:
# 2_Feature_Engineering (Step 2)
# - Read Step 1 output: Data/Processing/processed.parquet
# - Build features on full data, then split train/test by date_on
# - Output to Data/Feature/ (used by Step 3)


In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

RISK_CATEGORIES = ["heat_stress", "unseasonably_cold", "excess_precip", "drought"]

# Local / Kaggle compatibility
if Path("/kaggle/input").exists():
    DATA_DIR = Path("/kaggle/input/forecasting-the-future-the-helios-corn-climate-challenge")
    OUT_DIR = Path("/kaggle/working")
else:
    BASE_DIR = Path.cwd()
    DATA_DIR = BASE_DIR / "forecasting-the-future-the-helios-corn-climate-challenge"
    OUT_DIR = BASE_DIR

PROC_PATH = OUT_DIR / "Data" / "Processing" / "processed.parquet"
SHARE_PATH = DATA_DIR / "corn_regional_market_share.csv"

FEAT_DIR = OUT_DIR / "Data" / "Feature"
FEAT_DIR.mkdir(parents=True, exist_ok=True)

print("PROC_PATH:", PROC_PATH)
print("SHARE_PATH:", SHARE_PATH)
print("FEAT_DIR:", FEAT_DIR)


PROC_PATH: e:\Desktop\Mr RRR_Helios Corn Futures Climate Challenge_Repo\Data\Processing\processed.parquet
SHARE_PATH: e:\Desktop\Mr RRR_Helios Corn Futures Climate Challenge_Repo\forecasting-the-future-the-helios-corn-climate-challenge\corn_regional_market_share.csv
FEAT_DIR: e:\Desktop\Mr RRR_Helios Corn Futures Climate Challenge_Repo\Data\Feature


In [3]:
# Load processed data + market share

df = pd.read_parquet(PROC_PATH)
df["date_on"] = pd.to_datetime(df["date_on"], errors="coerce")

market_share_df = pd.read_csv(SHARE_PATH, low_memory=False)

print("processed df:", df.shape)

# Merge percent_country_production (fill missing with 1.0)
df = df.merge(market_share_df[["region_id", "percent_country_production"]], on="region_id", how="left")
df["percent_country_production"] = df["percent_country_production"].fillna(1.0)


processed df: (320447, 41)


In [4]:
# Feature engineering

def build_features(base: pd.DataFrame, seasonal_stats: pd.DataFrame | None = None, thresholds: dict | None = None):
    df = base.copy()
    new_cols = []

    # Collect features first, then concatenate once to avoid DataFrame fragmentation
    feats: dict[str, pd.Series] = {}

    # Time features
    feats["climate_risk_day_of_year"] = df["date_on"].dt.dayofyear
    feats["climate_risk_quarter"] = df["date_on"].dt.quarter
    feats["climate_risk_season_sin"] = np.sin(2 * np.pi * feats["climate_risk_day_of_year"] / 365.0)
    feats["climate_risk_season_cos"] = np.cos(2 * np.pi * feats["climate_risk_day_of_year"] / 365.0)

    # Hemisphere-aligned seasonal features & key window weights
    south_countries = {
        "Brazil",
        "Argentina",
        "Australia",
        "South Africa",
        "Uruguay",
        "Paraguay",
        "Bolivia",
        "Chile",
        "Peru",
        "New Zealand",
    }
    is_south = df["country_name"].isin(south_countries)
    month = df["date_on"].dt.month
    month_adj = np.where(is_south, ((month + 5) % 12) + 1, month)
    season_weight = np.where(
        np.isin(month_adj, [7, 8]),
        1.5,
        np.where(np.isin(month_adj, [6, 9]), 1.2, 1.0),
    )
    feats["climate_risk_season_weight"] = season_weight

    doy_shift = np.where(is_south, 182, 0)
    doy_shifted = (feats["climate_risk_day_of_year"] + doy_shift) % 365
    feats["climate_risk_season_sin_shift"] = np.sin(2 * np.pi * doy_shifted / 365.0)
    feats["climate_risk_season_cos_shift"] = np.cos(2 * np.pi * doy_shifted / 365.0)

    # Base risk scores + production weighting
    for risk_type in RISK_CATEGORIES:
        low_col = f"climate_risk_cnt_locations_{risk_type}_risk_low"
        med_col = f"climate_risk_cnt_locations_{risk_type}_risk_medium"
        high_col = f"climate_risk_cnt_locations_{risk_type}_risk_high"

        total = df[low_col] + df[med_col] + df[high_col]
        score = (df[med_col] + 2.0 * df[high_col]) / (total + 1e-6)
        high_share = df[high_col] / (total + 1e-6)
        med_share = df[med_col] / (total + 1e-6)
        low_share = df[low_col] / (total + 1e-6)
        severity_051 = (0.5 * df[med_col] + 1.0 * df[high_col]) / (total + 1e-6)
        balance = (df[high_col] - df[low_col]) / (total + 1e-6)
        entropy = -(
            low_share * np.log(low_share + 1e-6)
            + med_share * np.log(med_share + 1e-6)
            + high_share * np.log(high_share + 1e-6)
        )

        weight = df["percent_country_production"] / 100.0
        weighted = score * weight
        weighted_high = high_share * weight
        weighted_med = med_share * weight
        weighted_sev051 = severity_051 * weight
        weighted_balance = balance * weight

        feats[f"climate_risk_{risk_type}_score"] = score
        feats[f"climate_risk_{risk_type}_weighted"] = weighted
        feats[f"climate_risk_{risk_type}_high_share"] = high_share
        feats[f"climate_risk_{risk_type}_med_share"] = med_share
        feats[f"climate_risk_{risk_type}_low_share"] = low_share
        feats[f"climate_risk_{risk_type}_severity_051"] = severity_051
        feats[f"climate_risk_{risk_type}_balance"] = balance
        feats[f"climate_risk_{risk_type}_entropy"] = entropy
        feats[f"climate_risk_{risk_type}_weighted_high_share"] = weighted_high
        feats[f"climate_risk_{risk_type}_weighted_med_share"] = weighted_med
        feats[f"climate_risk_{risk_type}_weighted_severity_051"] = weighted_sev051
        feats[f"climate_risk_{risk_type}_weighted_balance"] = weighted_balance

    # Concatenate base features once
    base_feat_df = pd.DataFrame(feats, index=df.index)
    df = df.join(base_feat_df)
    new_cols.extend(list(base_feat_df.columns))

    # Sort for time-series features (by region_id)
    df = df.sort_values(["region_id", "date_on"]).copy()

    # Rolling / Lag / EMA / Momentum
    roll_feats: dict[str, pd.Series] = {}
    ROLL_WINDOWS = [7, 14, 30, 60, 90]
    LAGS = [1, 7, 14, 30]
    EMAS = [7, 14, 30, 60]

    for window in ROLL_WINDOWS:
        for risk_type in RISK_CATEGORIES:
            score_col = f"climate_risk_{risk_type}_score"

            roll_feats[f"climate_risk_{risk_type}_ma_{window}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.rolling(window, min_periods=1).mean())
            )
            roll_feats[f"climate_risk_{risk_type}_max_{window}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.rolling(window, min_periods=1).max())
            )
            roll_feats[f"climate_risk_{risk_type}_vol_{window}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.rolling(window, min_periods=2).std())
            )
            roll_feats[f"climate_risk_{risk_type}_cumsum_{window}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.rolling(window, min_periods=1).sum())
            )

    for lag in LAGS:
        for risk_type in RISK_CATEGORIES:
            score_col = f"climate_risk_{risk_type}_score"
            roll_feats[f"climate_risk_{risk_type}_lag_{lag}d"] = df.groupby("region_id")[score_col].shift(lag)

    for span in EMAS:
        for risk_type in RISK_CATEGORIES:
            score_col = f"climate_risk_{risk_type}_score"
            roll_feats[f"climate_risk_{risk_type}_ema_{span}d"] = (
                df.groupby("region_id")[score_col].transform(lambda x: x.ewm(span=span, min_periods=1).mean())
            )

    # Momentum / acceleration
    for risk_type in RISK_CATEGORIES:
        score_col = f"climate_risk_{risk_type}_score"
        c1 = df.groupby("region_id")[score_col].diff(1)
        roll_feats[f"climate_risk_{risk_type}_change_1d"] = c1
        roll_feats[f"climate_risk_{risk_type}_change_7d"] = df.groupby("region_id")[score_col].diff(7)
        roll_feats[f"climate_risk_{risk_type}_acceleration"] = c1.groupby(df["region_id"]).diff(1)

        lag7 = df.groupby("region_id")[score_col].shift(7)
        roll_feats[f"climate_risk_{risk_type}_roc_7d"] = (df[score_col] - lag7) / (lag7 + 1e-6)

    # Event and persistence features
    EVENT_TAUS = [0.2, 0.3]
    EVENT_WINDOWS = [7, 14, 30]
    SPIKE_WINDOW = 30
    SPIKE_Z = 2.0

    for risk_type in RISK_CATEGORIES:
        hs_col = f"climate_risk_{risk_type}_high_share"

        for tau in EVENT_TAUS:
            runlen = df.groupby("region_id")[hs_col].transform(
                lambda s: (s > tau).groupby((~(s > tau)).cumsum()).cumsum()
            )
            roll_feats[f"climate_risk_{risk_type}_high_runlen_{int(tau*100)}"] = runlen

            time_since = df.groupby("region_id")[hs_col].transform(
                lambda s: (~(s > tau)).groupby((s > tau).cumsum()).cumcount()
            )
            roll_feats[f"climate_risk_{risk_type}_time_since_high_{int(tau*100)}"] = time_since

            for window in EVENT_WINDOWS:
                auc = df.groupby("region_id")[hs_col].transform(
                    lambda s: (s - tau).clip(lower=0).rolling(window, min_periods=1).sum()
                )
                roll_feats[f"climate_risk_{risk_type}_event_auc_{int(tau*100)}_{window}d"] = auc

        mean_ = df.groupby("region_id")[hs_col].transform(lambda s: s.rolling(SPIKE_WINDOW, min_periods=2).mean())
        std_ = df.groupby("region_id")[hs_col].transform(lambda s: s.rolling(SPIKE_WINDOW, min_periods=2).std())
        roll_feats[f"climate_risk_{risk_type}_spike_{SPIKE_WINDOW}d"] = (
            (df[hs_col] - mean_) > (SPIKE_Z * std_)
        ).astype(int)

    roll_feat_df = pd.DataFrame(roll_feats, index=df.index)
    df = df.join(roll_feat_df)
    new_cols.extend(list(roll_feat_df.columns))

    # Nonlinear and combined features
    combo_feats: dict[str, pd.Series] = {}
    score_cols = [f"climate_risk_{r}_score" for r in RISK_CATEGORIES]
    weighted_cols = [f"climate_risk_{r}_weighted" for r in RISK_CATEGORIES]

    combo_feats["climate_risk_temperature_stress"] = df[["climate_risk_heat_stress_score", "climate_risk_unseasonably_cold_score"]].max(axis=1)
    combo_feats["climate_risk_precipitation_stress"] = df[["climate_risk_excess_precip_score", "climate_risk_drought_score"]].max(axis=1)
    combo_feats["climate_risk_overall_stress"] = df[score_cols].max(axis=1)
    combo_feats["climate_risk_combined_stress"] = df[score_cols].mean(axis=1)

    combo_feats["climate_risk_temperature_stress_weighted"] = df[["climate_risk_heat_stress_weighted", "climate_risk_unseasonably_cold_weighted"]].max(axis=1)
    combo_feats["climate_risk_precipitation_stress_weighted"] = df[["climate_risk_excess_precip_weighted", "climate_risk_drought_weighted"]].max(axis=1)
    combo_feats["climate_risk_overall_weighted"] = df[weighted_cols].max(axis=1)
    combo_feats["climate_risk_combined_weighted"] = df[weighted_cols].mean(axis=1)

    combo_feats["climate_risk_precip_drought_diff"] = df["climate_risk_excess_precip_score"] - df["climate_risk_drought_score"]
    combo_feats["climate_risk_temp_diff"] = df["climate_risk_heat_stress_score"] - df["climate_risk_unseasonably_cold_score"]
    combo_feats["climate_risk_precip_drought_ratio"] = df["climate_risk_excess_precip_score"] / (df["climate_risk_drought_score"] + 0.01)

    combo_feats["climate_risk_heat_cold_sum"] = df["climate_risk_heat_stress_score"] + df["climate_risk_unseasonably_cold_score"]
    combo_feats["climate_risk_drought_excess_sum"] = df["climate_risk_drought_score"] + df["climate_risk_excess_precip_score"]
    combo_feats["climate_risk_heat_drought_interact"] = df["climate_risk_heat_stress_score"] * df["climate_risk_drought_score"]
    combo_feats["climate_risk_cold_excess_interact"] = df["climate_risk_unseasonably_cold_score"] * df["climate_risk_excess_precip_score"]

    # season/time modulation
    for risk_type in RISK_CATEGORIES:
        score_col = f"climate_risk_{risk_type}_score"
        weighted_col = f"climate_risk_{risk_type}_weighted"
        combo_feats[f"climate_risk_{risk_type}_score_season_sin"] = df[score_col] * df["climate_risk_season_sin"]
        combo_feats[f"climate_risk_{risk_type}_score_season_cos"] = df[score_col] * df["climate_risk_season_cos"]
        combo_feats[f"climate_risk_{risk_type}_weighted_season_sin"] = df[weighted_col] * df["climate_risk_season_sin"]
        combo_feats[f"climate_risk_{risk_type}_weighted_season_cos"] = df[weighted_col] * df["climate_risk_season_cos"]
        combo_feats[f"climate_risk_{risk_type}_score_qtr"] = df[score_col] * df["climate_risk_quarter"]
        combo_feats[f"climate_risk_{risk_type}_score_doy"] = df[score_col] * (df["climate_risk_day_of_year"] / 365.0)
        combo_feats[f"climate_risk_{risk_type}_season_w"] = df[score_col] * df["climate_risk_season_weight"]

    combo_df = pd.DataFrame(combo_feats, index=df.index)
    df = df.join(combo_df)
    new_cols.extend(list(combo_df.columns))

    # Seasonal anomalies (z-score by country_name + month)
    df["month"] = df["date_on"].dt.month
    if seasonal_stats is None:
        seasonal_stats = (
            df.groupby(["country_name", "month"]) 
            .agg(**{f"{r}_mean": (f"climate_risk_{r}_score", "mean") for r in RISK_CATEGORIES},
                 **{f"{r}_std": (f"climate_risk_{r}_score", "std") for r in RISK_CATEGORIES})
            .reset_index()
        )

    df = df.merge(seasonal_stats, on=["country_name", "month"], how="left")

    z_feats: dict[str, pd.Series] = {}
    for r in RISK_CATEGORIES:
        mean_c = f"{r}_mean"
        std_c = f"{r}_std"
        z_feats[f"climate_risk_{r}_seasonal_z"] = (df[f"climate_risk_{r}_score"] - df[mean_c]) / (df[std_c] + 1e-6)

    z_df = pd.DataFrame(z_feats, index=df.index)
    df = df.join(z_df)
    new_cols.extend(list(z_df.columns))

    # Extreme risk thresholds (computed from train)
    if thresholds is None:
        thresholds = {}
        for r in RISK_CATEGORIES:
            thresholds[r] = df[f"climate_risk_{r}_high_share"].quantile(0.9)

    flag_feats: dict[str, pd.Series] = {}
    for r in RISK_CATEGORIES:
        thr = thresholds[r]
        flag_feats[f"climate_risk_{r}_extreme"] = (df[f"climate_risk_{r}_high_share"] >= thr).astype(int)

    flag_df = pd.DataFrame(flag_feats, index=df.index)
    df = df.join(flag_df)
    new_cols.extend(list(flag_df.columns))

    # NaN handling (one pass)
    df[new_cols] = df[new_cols].fillna(0)

    return df, seasonal_stats, thresholds, new_cols


In [5]:
# Fit feature parameters on train, then apply to test

def add_country_agg(df: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    country_feats = []
    for risk_type in RISK_CATEGORIES:
        score_col = f"climate_risk_{risk_type}_score"
        weighted_col = f"climate_risk_{risk_type}_weighted"

        country_agg = (
            df.groupby(["country_name", "date_on"]) 
            .agg(
                **{
                    f"climate_risk_country_{risk_type}_score_mean": (score_col, "mean"),
                    f"climate_risk_country_{risk_type}_score_max": (score_col, "max"),
                    f"climate_risk_country_{risk_type}_score_std": (score_col, "std"),
                    f"climate_risk_country_{risk_type}_weighted_sum": (weighted_col, "sum"),
                }
            )
            .reset_index()
        )

        base_cols = [c for c in country_agg.columns if c not in ["country_name", "date_on"]]
        country_feats.extend(base_cols)
        df = df.merge(country_agg, on=["country_name", "date_on"], how="left")

        high_share_col = f"climate_risk_{risk_type}_high_share"

        def _country_metrics(g: pd.DataFrame) -> pd.Series:
            w = g["percent_country_production"].fillna(0) / 100.0
            x = g[score_col]
            hs = g[high_share_col]
            w_sum = w.sum()

            if w_sum > 0:
                w_mean = np.average(x, weights=w)
                w_std = np.sqrt(np.average((x - w_mean) ** 2, weights=w))
            else:
                w_mean = x.mean()
                w_std = x.std(ddof=0)

            s = w * x
            denom = (s.sum() ** 2) + 1e-6
            hhi = (s ** 2).sum() / denom
            exposed = (w * (hs > 0.3)).sum()

            if s.size <= 1:
                top_diff = float(s.max()) if s.size == 1 else 0.0
            else:
                s_sorted = np.sort(s.to_numpy())
                top_diff = float(s_sorted[-1] - s_sorted[-2])

            return pd.Series(
                {
                    f"climate_risk_country_{risk_type}_score_wmean": w_mean,
                    f"climate_risk_country_{risk_type}_score_wstd": w_std,
                    f"climate_risk_country_{risk_type}_concentration_hhi": hhi,
                    f"climate_risk_country_{risk_type}_prod_exposed_share": exposed,
                    f"climate_risk_country_{risk_type}_top1_minus_top2": top_diff,
                }
            )

        extra = (
            df.groupby(["country_name", "date_on"], sort=False)
            .apply(_country_metrics)
            .reset_index()
        )

        extra_cols = [c for c in extra.columns if c not in ["country_name", "date_on"]]
        country_feats.extend(extra_cols)
        df = df.merge(extra, on=["country_name", "date_on"], how="left")

    # NaN handling
    df[country_feats] = df[country_feats].fillna(0)
    return df, country_feats

# Build features on full data first

a_all, seasonal_stats, thresholds, all_feat_cols = build_features(df, seasonal_stats=None, thresholds=None)
a_all, country_cols_all = add_country_agg(a_all)

print("all features:", len(all_feat_cols) + len(country_cols_all))

# Time split: 80/20 by unique date_on sequence
all_dates = pd.Series(a_all["date_on"].dropna().unique()).sort_values().reset_index(drop=True)
cut = int(len(all_dates) * 0.8)
train_dates = set(all_dates.iloc[:cut])
test_dates = set(all_dates.iloc[cut:])

a_train = a_all[a_all["date_on"].isin(train_dates)].copy()
a_test = a_all[a_all["date_on"].isin(test_dates)].copy()

print("train:", a_train.shape, "test:", a_test.shape)
print("train date range:", a_train["date_on"].min(), "~", a_train["date_on"].max())
print("test  date range:", a_test["date_on"].min(), "~", a_test["date_on"].max())


all features: 314
train: (255756, 365) test: (64691, 365)
train date range: 2016-01-04 00:00:00 ~ 2023-12-19 00:00:00
test  date range: 2023-12-20 00:00:00 ~ 2025-12-15 00:00:00


In [6]:
# Save train / test to Data/Feature/

train_path = FEAT_DIR / "train.parquet"
test_path = FEAT_DIR / "test.parquet"

a_train.to_parquet(train_path, index=False)
a_test.to_parquet(test_path, index=False)

print("saved:", train_path)
print("saved:", test_path)

# Save climate feature list (based on train)
all_climate_cols = [c for c in a_train.columns if c.startswith("climate_risk_")]
feat_list_path = FEAT_DIR / "climate_feature_columns.parquet"
pd.DataFrame({"feature": all_climate_cols}).to_parquet(feat_list_path, index=False)
print("saved:", feat_list_path, "| n_climate_features:", len(all_climate_cols))

saved: e:\Desktop\Mr RRR_Helios Corn Futures Climate Challenge_Repo\Data\Feature\train.parquet
saved: e:\Desktop\Mr RRR_Helios Corn Futures Climate Challenge_Repo\Data\Feature\test.parquet
saved: e:\Desktop\Mr RRR_Helios Corn Futures Climate Challenge_Repo\Data\Feature\climate_feature_columns.parquet | n_climate_features: 326


In [7]:
a_train

,ID,crop_name,country_name,country_code,region_name,region_id,harvest_period,growing_season_year,date_on,climate_risk_cnt_locations_heat_stress_risk_low,climate_risk_cnt_locations_heat_stress_risk_medium,climate_risk_cnt_locations_heat_stress_risk_high,climate_risk_cnt_locations_unseasonably_cold_risk_low,climate_risk_cnt_locations_unseasonably_cold_risk_medium,climate_risk_cnt_locations_unseasonably_cold_risk_high,climate_risk_cnt_locations_excess_precip_risk_low,climate_risk_cnt_locations_excess_precip_risk_medium,climate_risk_cnt_locations_excess_precip_risk_high,climate_risk_cnt_locations_drought_risk_low,climate_risk_cnt_locations_drought_risk_medium,climate_risk_cnt_locations_drought_risk_high,futures_close_ZC_1,futures_close_ZC_2,futures_close_ZW_1,futures_close_ZS_1,futures_zc1_ret_pct,futures_zc1_ret_log,futures_zc_term_spread,futures_zc_term_ratio,futures_zc1_ma_20,futures_zc1_ma_60,futures_zc1_ma_120,futures_zc1_vol_20,futures_zc1_vol_60,futures_zw_zc_spread,futures_zc_zw_ratio,futures_zs_zc_spread,futures_zc_zs_ratio,date_on_year,date_on_month,date_on_year_month,percent_country_production,climate_risk_day_of_year,climate_risk_quarter,climate_risk_season_sin,climate_risk_season_cos,climate_risk_season_weight,climate_risk_season_sin_shift,climate_risk_season_cos_shift,climate_risk_heat_stress_score,climate_risk_heat_stress_weighted,climate_risk_heat_stress_high_share,climate_risk_heat_stress_med_share,climate_risk_heat_stress_low_share,climate_risk_heat_stress_severity_051,climate_risk_heat_stress_balance,climate_risk_heat_stress_entropy,climate_risk_heat_stress_weighted_high_share,climate_risk_heat_stress_weighted_med_share,climate_risk_heat_stress_weighted_severity_051,climate_risk_heat_stress_weighted_balance,climate_risk_unseasonably_cold_score,climate_risk_unseasonably_cold_weighted,climate_risk_unseasonably_cold_high_share,climate_risk_unseasonably_cold_med_share,climate_risk_unseasonably_cold_low_share,climate_risk_unseasonably_cold_severity_051,climate_risk_unseasonably_cold_balance,climate_risk_unseasonably_cold_entropy,climate_risk_unseasonably_cold_weighted_high_share,climate_risk_unseasonably_cold_weighted_med_share,climate_risk_unseasonably_cold_weighted_severity_051,climate_risk_unseasonably_cold_weighted_balance,climate_risk_excess_precip_score,climate_risk_excess_precip_weighted,climate_risk_excess_precip_high_share,climate_risk_excess_precip_med_share,climate_risk_excess_precip_low_share,climate_risk_excess_precip_severity_051,climate_risk_excess_precip_balance,climate_risk_excess_precip_entropy,climate_risk_excess_precip_weighted_high_share,climate_risk_excess_precip_weighted_med_share,climate_risk_excess_precip_weighted_severity_051,climate_risk_excess_precip_weighted_balance,climate_risk_drought_score,climate_risk_drought_weighted,climate_risk_drought_high_share,climate_risk_drought_med_share,climate_risk_drought_low_share,climate_risk_drought_severity_051,climate_risk_drought_balance,climate_risk_drought_entropy,climate_risk_drought_weighted_high_share,climate_risk_drought_weighted_med_share,climate_risk_drought_weighted_severity_051,climate_risk_drought_weighted_balance,climate_risk_heat_stress_ma_7d,climate_risk_heat_stress_max_7d,climate_risk_heat_stress_vol_7d,...,climate_risk_drought_event_auc_30_7d,climate_risk_drought_event_auc_30_14d,climate_risk_drought_event_auc_30_30d,climate_risk_drought_spike_30d,climate_risk_temperature_stress,climate_risk_precipitation_stress,climate_risk_overall_stress,climate_risk_combined_stress,climate_risk_temperature_stress_weighted,climate_risk_precipitation_stress_weighted,climate_risk_overall_weighted,climate_risk_combined_weighted,climate_risk_precip_drought_diff,climate_risk_temp_diff,climate_risk_precip_drought_ratio,climate_risk_heat_cold_sum,climate_risk_drought_excess_sum,climate_risk_heat_drought_interact,climate_risk_cold_excess_interact,climate_risk_heat_stress_score_season_sin,climate_risk_heat_stress_score_season_cos,climate_risk_heat_stress_wei

In [8]:
a_test

,ID,crop_name,country_name,country_code,region_name,region_id,harvest_period,growing_season_year,date_on,climate_risk_cnt_locations_heat_stress_risk_low,climate_risk_cnt_locations_heat_stress_risk_medium,climate_risk_cnt_locations_heat_stress_risk_high,climate_risk_cnt_locations_unseasonably_cold_risk_low,climate_risk_cnt_locations_unseasonably_cold_risk_medium,climate_risk_cnt_locations_unseasonably_cold_risk_high,climate_risk_cnt_locations_excess_precip_risk_low,climate_risk_cnt_locations_excess_precip_risk_medium,climate_risk_cnt_locations_excess_precip_risk_high,climate_risk_cnt_locations_drought_risk_low,climate_risk_cnt_locations_drought_risk_medium,climate_risk_cnt_locations_drought_risk_high,futures_close_ZC_1,futures_close_ZC_2,futures_close_ZW_1,futures_close_ZS_1,futures_zc1_ret_pct,futures_zc1_ret_log,futures_zc_term_spread,futures_zc_term_ratio,futures_zc1_ma_20,futures_zc1_ma_60,futures_zc1_ma_120,futures_zc1_vol_20,futures_zc1_vol_60,futures_zw_zc_spread,futures_zc_zw_ratio,futures_zs_zc_spread,futures_zc_zs_ratio,date_on_year,date_on_month,date_on_year_month,percent_country_production,climate_risk_day_of_year,climate_risk_quarter,climate_risk_season_sin,climate_risk_season_cos,climate_risk_season_weight,climate_risk_season_sin_shift,climate_risk_season_cos_shift,climate_risk_heat_stress_score,climate_risk_heat_stress_weighted,climate_risk_heat_stress_high_share,climate_risk_heat_stress_med_share,climate_risk_heat_stress_low_share,climate_risk_heat_stress_severity_051,climate_risk_heat_stress_balance,climate_risk_heat_stress_entropy,climate_risk_heat_stress_weighted_high_share,climate_risk_heat_stress_weighted_med_share,climate_risk_heat_stress_weighted_severity_051,climate_risk_heat_stress_weighted_balance,climate_risk_unseasonably_cold_score,climate_risk_unseasonably_cold_weighted,climate_risk_unseasonably_cold_high_share,climate_risk_unseasonably_cold_med_share,climate_risk_unseasonably_cold_low_share,climate_risk_unseasonably_cold_severity_051,climate_risk_unseasonably_cold_balance,climate_risk_unseasonably_cold_entropy,climate_risk_unseasonably_cold_weighted_high_share,climate_risk_unseasonably_cold_weighted_med_share,climate_risk_unseasonably_cold_weighted_severity_051,climate_risk_unseasonably_cold_weighted_balance,climate_risk_excess_precip_score,climate_risk_excess_precip_weighted,climate_risk_excess_precip_high_share,climate_risk_excess_precip_med_share,climate_risk_excess_precip_low_share,climate_risk_excess_precip_severity_051,climate_risk_excess_precip_balance,climate_risk_excess_precip_entropy,climate_risk_excess_precip_weighted_high_share,climate_risk_excess_precip_weighted_med_share,climate_risk_excess_precip_weighted_severity_051,climate_risk_excess_precip_weighted_balance,climate_risk_drought_score,climate_risk_drought_weighted,climate_risk_drought_high_share,climate_risk_drought_med_share,climate_risk_drought_low_share,climate_risk_drought_severity_051,climate_risk_drought_balance,climate_risk_drought_entropy,climate_risk_drought_weighted_high_share,climate_risk_drought_weighted_med_share,climate_risk_drought_weighted_severity_051,climate_risk_drought_weighted_balance,climate_risk_heat_stress_ma_7d,climate_risk_heat_stress_max_7d,climate_risk_heat_stress_vol_7d,...,climate_risk_drought_event_auc_30_7d,climate_risk_drought_event_auc_30_14d,climate_risk_drought_event_auc_30_30d,climate_risk_drought_spike_30d,climate_risk_temperature_stress,climate_risk_precipitation_stress,climate_risk_overall_stress,climate_risk_combined_stress,climate_risk_temperature_stress_weighted,climate_risk_precipitation_stress_weighted,climate_risk_overall_weighted,climate_risk_combined_weighted,climate_risk_precip_drought_diff,climate_risk_temp_diff,climate_risk_precip_drought_ratio,climate_risk_heat_cold_sum,climate_risk_drought_excess_sum,climate_risk_heat_drought_interact,climate_risk_cold_excess_interact,climate_risk_heat_stress_score_season_sin,climate_risk_heat_stress_score_season_cos,climate_risk_heat_stress_wei